<a href="https://colab.research.google.com/github/pandov/cups-mail/blob/master/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
! sh colab.sh

In [1]:
%%capture
import torch
from src.nn import BACTERIA, Runner, get_classification_components, get_loaders, score_clf
logdir = './logs/classification'
! rm -r {logdir}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
class MultiRunner(Runner):
    def _handle_batch(self, batch):
        x, y = batch
        y_pred = self.model(x)
        is_train = self.is_train_loader == True

        with torch.set_grad_enabled(is_train):
            loss = self.criterion(y_pred, y)
            precision = score_clf(y_pred, y)
            self.batch_metrics.update({
                'precision/_mean': precision.mean(),
                'precision/_sum': precision.sum(),
                'loss': loss,
                'lr': self.state.scheduler.get_last_lr()[0],
            })

            if is_train:
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
            else:
                if self.loader_batch_step == self.loader_len:
                    self.scheduler.step()
        
        self.input = {'targets': y}
        self.output = {'logits': y_pred}

In [2]:
net = 'efficientnet-b4'
components = get_classification_components(net, 'adam', 'steplr', weightable=True)
print(components['model'])
# components['loaders'] = get_loaders(batch_size=10, keys=['image', 'label'], is_resized=True, is_negative=False)
# runner = MultiRunner()
# runner.train(
#     logdir=f'{logdir}/{net}',
#     num_epochs=140,
#     # verbose=True,
#     **components
# )

=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): BatchNorm2d(112, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_swish): MemoryEfficientSwish()
    )
    (16): MBConvBlock(
      (_expand_conv): Conv2dStaticSamePadding(
        112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn0): BatchNorm2d(672, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_depthwise_conv): Conv2dStaticSamePadding(
        672, 672, kernel_size=(5, 5), stride=[1, 1], groups=672, bias=False
        (static_padding): ZeroPad2d(padding=(2, 2, 2, 2), value=0.0)
      )
      (_bn1): BatchNorm2d(672, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        672, 28, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expan

In [ ]:
logzip = logdir + '.zip'
! zip -r {logzip} {logdir}
! mv {logzip} 'drive/My Drive/'
# ! git pull origin master
# ! git add logs
# ! git commit -m 'Changed from Colab'
# ! git push -u origin master